In [9]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder \
    .appName("count") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "1024M") \
    .getOrCreate()
sc = spark.sparkContext

### Question 1



In [10]:
df = spark.read.json("hdfs://localhost:9000/datasets/spotify/tracks.json")
df.printSchema()
df.show(5,False)

root
 |-- album_name: string (nullable = true)
 |-- album_uri: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_uri: string (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- pid: long (nullable = true)
 |-- pos: long (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_uri: string (nullable = true)

+--------------------------------------------+------------------------------------+-----------------+-------------------------------------+-----------+---+---+------------------------------------------+------------------------------------+
|album_name                                  |album_uri                           |artist_name      |artist_uri                           |duration_ms|pid|pos|track_name                                |track_uri                           |
+--------------------------------------------+------------------------------------+-----------------+-------------------------------------+-----------+---+-

#### 1.1

In [11]:
df1_1 = df.select("duration_ms").summary("min","max","mean")
df1_1.show()

+-------+------------------+
|summary|       duration_ms|
+-------+------------------+
|    min|                 0|
|    max|          10435467|
|   mean|234408.54976216817|
+-------+------------------+



#### 1.2&1.3

In [12]:
import pandas as pd
df1_2 = df.select("duration_ms").summary("25%", "75%")
df1_2.show()

pd1_2 = df1_2.toPandas()
Q1 = int(pd1_2["duration_ms"][0])
Q3 = int(pd1_2["duration_ms"][1])
print("Q1 = {0}".format(Q1))
print("Q3 = {0}".format(Q3))
IQR = Q3 - Q1
print("IQR = {0}".format(IQR))

duration_ms_column = df["duration_ms"]
df_filter = df.filter(duration_ms_column < Q3 + 1.5 * IQR).filter(Q1 - 1.5 * IQR < duration_ms_column)
df_filter.show(5,False)


+-------+-----------+
|summary|duration_ms|
+-------+-----------+
|    25%|     198333|
|    75%|     258834|
+-------+-----------+

Q1 = 198333
Q3 = 258834
IQR = 60501
+--------------------------------------------+------------------------------------+-----------------+-------------------------------------+-----------+---+---+------------------------------------------+------------------------------------+
|album_name                                  |album_uri                           |artist_name      |artist_uri                           |duration_ms|pid|pos|track_name                                |track_uri                           |
+--------------------------------------------+------------------------------------+-----------------+-------------------------------------+-----------+---+---+------------------------------------------+------------------------------------+
|The Cookbook                                |spotify:album:6vV5UrXcfyQD1wu4Qo2I9K|Missy Elliott    |spotify:ar

#### 1.4

In [16]:
df_count = df.select("duration_ms").summary("count")
df_count.show()
df_filter_count = df_filter.select("duration_ms").summary("count","min","max","mean")
df_filter_count.show()


+-------+-----------+
|summary|duration_ms|
+-------+-----------+
|  count|   10760544|
+-------+-----------+



+-------+------------------+
|summary|       duration_ms|
+-------+------------------+
|  count|          10200555|
|    min|            107582|
|    max|            349583|
|   mean|226899.35353939075|
+-------+------------------+



In [17]:
print("the number of outliers = {0}".format(10760544-10200555))

the number of outliers = 559989
